# Milky Way Mapper's Galaxy

## Section 7: Paper Task

In [1]:
#Import some things
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import fits, ascii
from astropy.table import Table
from astropy.coordinates import SkyCoord
%matplotlib inline


In [2]:
#load in the data (may have to change this for wherever you downloaded your file)
#in google colab you can get the file using
#!wget https://dr19.sdss.org/sas/dr19/spectro/astra/0.6.0/summary/astraAllStarASPCAP-0.6.0.fits.gz 

filename='astraAllStarASPCAP-0.6.0.fits'
tb = fits.open(filename)
header=tb[2].header
data = tb[2].data 

In [3]:
good=np.where((data['teff'] > 3700) & (data['teff'] < 5300) &
               (data['logg'] > 0.9) & (data['logg'] < 3.3) &
               (data['m_h_atm'] > -2.0) & (data['m_h_atm'] < 0.6) &   
               (data['flag_bad']==False) )

data_masked=data[good]

## Training Sets

In [4]:
#TESS
tessraw = Table.read("Theodoridis2025.csv", format="ascii")
#this one has an age column in Gyr already so we're just going to rename it Age
tessraw['Final_age'].name='Age'
hasagetess=np.where((tessraw['Age']==tessraw['Age']) & (tessraw['Age']>0.1) &(tessraw['Flag']==0))
tess=tessraw[hasagetess]
tess

TIC,Star_type,Age,Î½max,Radius_gaia,Teff_xgboost,M_H_xgboost,Logg_xgboost,Logg_seis,E_Logg_seis,Mass_seis,E_Mass_seis,Initial_mass,Teff_rgb,Teff_rc,Median_age_rgb,Median_age_rc,E_lower_age_rgb,E_upper_age_rgb,E_lower_age_rc,E_upper_age_rc,Teff_diff,Flag
int64,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
347020604,Clump,4.933697638292032,32.2,10.8,4766.6,-0.161,2.432,2.4173752480453827,0.0366707608673319,1.1120077414967078,0.0588614326241161,1.2303580537284051,4573.560449427352,4592.554431111989,7.362373244,4.928036510093708,6.081092717,8.892029592,4.299934128259921,6.1356166056543815,174.04556888801108,0
365250045,RGB,5.919390103924665,33.0,10.8,4931.7,-0.342,2.455,2.4229857865111493,0.0357953759646522,1.126466677413624,0.056320640605,1.1264672739859862,4667.980210870163,4834.105313542321,5.948811656,4.27541564590489,5.120775603,7.010332121,3.7125897559954217,4.990341868284544,97.59468645767902,0
377058143,RGB,10.659761236332615,59.1,7.3,4732.1,-0.519,2.615,2.677206425366256,0.0648094481080465,0.9241388003056158,0.0471439463924816,0.9241384053448526,4809.243666602538,4845.904744096898,11.11556225,5.653288060595535,9.53022434,12.72313934,4.950738403099905,6.798479977799735,-113.8047440968976,0
347548024,RGB,4.2433496208314345,33.2,11.5,4634.4,-0.213,2.388,2.4218043131513705,0.0720751969086103,1.2737524436569616,0.0852469599851652,1.2737527853903323,4625.72134164397,4775.614677688445,4.196919598,3.1842561951173276,3.279495422,5.737266629,2.7003724708906987,3.926199760609352,-141.2146776884456,0
328321210,Clump,5.297976079237009,35.7,9.6,4982.0,-0.526,2.441,2.4671643779400427,0.0755464565388593,0.9853531626679212,0.0677079590734425,1.121116190405825,4737.2709053692715,4762.055765851048,9.131354832,5.481214180765681,6.619391066,11.17929356,4.57496878200937,6.826530999798273,219.9442341489521,0
328321103,RGB,3.76901613056093,34.5,11.4,4765.0,-0.277,2.533,2.438719284222579,0.0355087401732596,1.3014095644450476,0.0659512559797257,1.301409615256684,4669.883118807332,4814.012809659391,3.7183394,2.964010779639734,3.222621332,4.398228001,2.51085645776994,3.520642797490074,-49.01280965939077,0
328324062,Clump,2.303632505100647,41.3,11.3,4792.0,0.085,2.617,2.5205598637381987,0.0266418863144747,1.5438382675435354,0.0669944467135548,1.6114631436790468,4552.67752843955,4567.434071459611,2.700422808,2.2357112051063632,2.255954628,3.155200636,1.9961451072036724,2.638300639580447,224.565928540389,0
328400618,Clump,2.159728298323744,44.1,10.5,4910.2,-0.319,2.472,2.557435937807216,0.0739867137774788,1.4511074516675977,0.0926043469167527,1.5165343396975142,4767.690838544538,4778.114861413547,2.561181642,2.157582282377099,2.082416797,3.109146383,1.7850147949917456,2.6517500725099072,132.08513858645256,0
328255103,Clump,6.418514994888695,31.9,10.8,4686.6,0.019,2.433,2.406553604071016,0.0559847430029249,1.0846413740056715,0.0674371343640133,1.18744043140101,4471.252872082005,4487.596554271885,9.075683921,6.586728636664553,7.526636104,11.69375071,5.35273124007062,7.587960124712628,199.00344572811537,0


In [5]:
# Age option 2 APOKASC-2 Pinsonneault et al. 2018
#Reading in the table, making sure all the tables have a column named Age in Gyr
# and that every star in the table has an Age
apokasc2raw = Table.read("Pinsonneault2018.txt", format="ascii.cds")
apokasc2raw['Age']=(10**np.array(apokasc2raw['LogAge'])/1000.) #Age was in log(Myr) so needs converting
hasagea2=np.where((apokasc2raw['Age']==apokasc2raw['Age']) & (apokasc2raw['Age']>0.1))
apokasc2=apokasc2raw[hasagea2]
apokasc2

KIC,2MASS,Teff,e_Teff,FeH,e_FeH,AFe,e_AFe,Nmax,e_Nmax,Dnu,e_Dnu,ES,Fdnu,e_Fdnu,M(cor),e_M(cor)-ran,e_M(cor)-sys,R(cor),e_R(cor)-ran,e_R(cor)-sys,logg(seis),e_logg(seis)-ran,e_logg(seis)-sys,Rho,e_Rho-ran,e_Rho-sys,LogAge,E_LogAge,e_LogAge,Av,e_Av,Notes,Age
,,K,K,dex(---),dex(---),dex(---),dex(---),uHz,,uHz,,,,,Msun,,,Rsun,,,dex(cm / s2),,,g / cm3,,,Myr,Myr,Myr,mag,mag,,
int64,str18,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str18,float64
1027110,2M19250937+3644599,4177.6,51.8,-0.232,0.025,0.205,0.015,6.496,0.015,1.132,0.029,RGB,1.0458,0.0006,0.985,0.126,0.044,23.412,0.06,0.017,1.692,0.007,0.007,0.0001082,0.058,0.008,4.002,0.205,-0.189,0.269,0.121,SeisUnc,10.046157902783946
1027337,2M19252021+3647118,4636.0,67.3,0.275,0.024,0.023,0.01,73.975,0.009,6.991,0.013,RGB,1.0333,0.0003,1.227,0.063,0.038,7.544,0.029,0.014,2.772,0.005,0.007,0.004029,0.026,0.007,3.798,0.099,-0.09,0.214,0.071,...,6.2805835881331795
1160789,2M19233280+3652207,4729.6,72.3,-0.257,0.034,0.188,0.015,25.209,0.015,3.545,0.011,RC,0.9965,0.0324,0.875,0.147,0.082,10.86,0.071,0.025,2.308,0.007,0.01,0.0009635,0.069,0.008,3.889,0.133,-0.146,0.009,0.074,...,7.744617978025183
1161447,2M19241746+3651460,4776.1,86.2,0.058,0.029,-0.006,0.013,37.066,0.027,4.153,0.011,RC,1.003,0.0238,1.46,0.135,0.076,11.54,0.059,0.022,2.478,0.012,0.01,0.00134,0.052,0.008,3.396,0.166,-0.145,0.4,0.086,...,2.48885731828239
1161618,2M19242614+3648478,4742.0,72.1,0.064,0.029,0.005,0.012,33.926,0.01,4.093,0.012,RC,1.001,0.0033,1.183,0.063,0.077,10.879,0.028,0.023,2.438,0.005,0.01,0.001296,0.026,0.008,3.639,0.064,-0.069,0.199,0.074,...,4.355118736855684
1162220,2M19245791+3653298,4190.1,51.7,0.083,0.021,0.07,0.011,11.0,0.01,1.669,0.011,RGB,1.0484,0.0004,1.007,0.055,0.044,18.175,0.024,0.017,1.922,0.005,0.007,0.0002364,0.021,0.008,4.056,0.087,-0.083,0.181,0.073,...,11.37627285823431
1162746,2M19252639+3649116,4798.1,75.6,-0.388,0.038,0.229,0.017,27.798,0.015,3.763,0.01,RC,0.9972,0.0281,0.941,0.131,0.08,10.688,0.062,0.024,2.354,0.007,0.01,0.001087,0.06,0.008,3.786,0.139,-0.131,0.172,0.075,...,6.109420249055721
1163114,2M19254564+3650475,4285.8,54.4,0.297,0.02,0.025,0.01,14.356,0.011,1.887,0.008,RGB,1.0396,0.0004,1.467,0.05,0.043,19.097,0.021,0.016,2.042,0.006,0.007,0.0002971,0.016,0.008,3.538,0.079,-0.077,0.382,0.069,...,3.451437393358561


In [6]:
# # Age option 3 APOKASC-3 Pinsonneault et al. 2025
# #Reading in the table, making sure all the tables have a column named Age in Gyr
# # and that every star in the table has an Age
# apokasc3raw= Table.read("Pinsonneault2025.txt", format="ascii.cds")
# #in this case there were two age columns, one for Red Clump and one for Red Giant Branch so we combine them
# ageRC=np.array(apokasc3raw['AgeRC']*(apokasc3raw['EvolState']=='RC'))
# rcnans=np.isnan(ageRC) #removing nans from this version of the table.
# ageRC[rcnans]=0
# ageRGB=np.array(apokasc3raw['AgeRGB']*(apokasc3raw['EvolState']=='RGB'))
# rgbnans=np.isnan(ageRGB) #removing nans from this version of the table.
# ageRGB[rgbnans]=0
# apokasc3raw['Age']=(ageRC+ageRGB)

# hasagea3=np.where((apokasc3raw['Age']==apokasc3raw['Age']) & (apokasc3raw['Age']>0.1))
# apokasc3=apokasc3raw[hasagea3]
# apokasc3

In [7]:
# # Age option 4 K2 data Warfield et al. 2024
# #Reading in the table, making sure all the tables have a column named Age in Gyr
# # and that every star in the table has an Age
# apok2raw = Table.read("Warfield2024.txt", format="ascii.cds")
# #this one has an age column in Gyr already so we're just going to rename it Age
# hasageapok2=np.where((apok2raw['Age']==apok2raw['Age']) & (apok2raw['Age']>0.1))
# apok2=apok2raw[hasageapok2]
# apok2

In [9]:
#My initial pick for training set
agedata= apokasc2
agedata2= tess

In [13]:
#Option 1 TESS Theodoridis et al. 2025
intersect2, ind_a2, ind_b2 = np.intersect1d(data_masked['tic_v8_id'],agedata2['TIC'], return_indices=True)

#Option 2 APOKASC-2 Pinsonneault et al. 2018
intersect, ind_a, ind_b = np.intersect1d(data_masked['sdss4_apogee_id'],agedata['2MASS'], return_indices=True) 

#Option 3 APOKASC-3 Pinsonneault et al. 2025
#intersect, ind_a, ind_b = np.intersect1d(data_masked['gaia_dr3_source_id'],agedata['GaiaDR3'], return_indices=True) 

#Option 4 APO-K2 Warfield et al. 2024
#intersect, ind_a, ind_b = np.intersect1d(data_masked['sdss4_apogee_id'],agedata['APOGEE'], return_indices=True) 

print(len(ind_b), len(ind_b2))

5993 15517


In [14]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [15]:
from tensorflow import keras

## First Training Set

In [18]:
fullx = np.dstack([data_masked['teff'][ind_a],data_masked['logg'][ind_a], data_masked['m_h_atm'][ind_a],
                   data_masked['alpha_m_atm'][ind_a], data_masked['c_h'][ind_a], data_masked['n_h'][ind_a]])[0]

fully = np.dstack([agedata['Age'][ind_b]])[0] #for Pinsonneault 2018

#remove non-finite entries!
mask = np.all(np.isfinite(fullx), axis=1) & np.all(np.isfinite(fully), axis=1)
fullx, fully = fullx[mask], fully[mask]

scaling_x = np.median(fullx, axis=0)
scaling_y = np.median(fully, axis=0)

fullx, fully = fullx/scaling_x, fully/scaling_y

In [19]:
neurons_per_layer=20
layers=5
iterations=200

In [20]:

#start with an input layer
inputs = keras.Input(shape=(6,))
#now we add the Dense layers (indicating the previous layer in the brackets following the layer declaration

#change this part if you're changing the number of layers
layer1 =keras.layers.Dense(neurons_per_layer, activation='relu')(inputs)
layer2 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer1)
layer3 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer2)
layer4 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer3)
layer5 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer4)
layer6 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer5)
layer7 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer6)
layer8 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer7)
layer9 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer8)
layer10 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer9)
layer11 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer10)
layer12 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer11)
layer13 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer12)
layer14 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer13)
layer15 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer14)
layer16 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer15)
layer17 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer16)
layer18 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer17)
layer19 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer18)
layer20 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer19)
layer21 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer20)
layer22 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer21)
layer23 = keras.layers.Dense(neurons_per_layer, activation='relu')(layer22)

#then the output layer YOU ALSO HAVE TO MAKE THIS MATCH YOUR NUMBER OF LAYERS
outputs = keras.layers.Dense(1)(layer4)


# then we put that all together in the Model object
model = keras.Model(inputs=inputs, outputs=outputs, name='test')
#and we can print a summary to check it all went to plan
model.summary()

Model: "test"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 6)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 20)                  │             140 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,421 (5.55 KB)

 Trainable params: 1,421 (5.55 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [22]:
tenpercent=len(agedata['Age'][ind_b])//10 #figure out what ten percent of this set of age data is

#last name before M 
#trainbin=slice(0,-1*tenpercent-1)
#testing=slice(-1*tenpercent,-1)


#last name M or later
trainbin=slice(tenpercent+1,-1)
testing=slice(0,tenpercent)


x_train, y_train = fullx[trainbin], fully[trainbin]
x_test, y_test = fullx[testing], fully[testing]

In [23]:
model.fit(x_train, y_train, epochs=iterations, validation_split=0.05, batch_size=300)

Epoch 1/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 2.0354e-04 - loss: 1.5948 - val_accuracy: 0.0000e+00 - val_loss: 0.8921
Epoch 2/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 6.1062e-04 - loss: 0.8438 - val_accuracy: 0.0000e+00 - val_loss: 0.6532
Epoch 3/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 6.1062e-04 - loss: 0.6393 - val_accuracy: 0.0000e+00 - val_loss: 0.5009
Epoch 4/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0010 - loss: 0.4901 - val_accuracy: 0.0000e+00 - val_loss: 0.3903
Epoch 5/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0012 - loss: 0.3868 - val_accuracy: 0.0039 - val_loss: 0.3205
Epoch 6/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0012 - loss: 0.3333 - val_accuracy: 0.0039 - val_loss: 0.2794
Epoch 7/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0012 - loss: 0.2991 - val_accuracy: 0.0039 - val_loss: 0.2503
Epoch 8/200
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0012 - los

In [24]:
predictions = model.predict(x_test)
print(len(predictions))

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
599


In [25]:
DR19x = np.dstack([data_masked['teff'],data_masked['logg'], data_masked['m_h_atm'],
                   data_masked['alpha_m_atm'], data_masked['c_h'], data_masked['n_h']])[0]
print(len(data_masked['teff']))

DR19x= DR19x/scaling_x
predictionsDR19 = model.predict(DR19x)

425631
13301/13301 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step


## Training Set 2

In [26]:
fullx2 = np.dstack([data_masked['teff'][ind_a2],data_masked['logg'][ind_a2], data_masked['m_h_atm'][ind_a2],
                   data_masked['alpha_m_atm'][ind_a2], data_masked['c_h'][ind_a2], data_masked['n_h'][ind_a2]])[0]

fully2 = np.dstack([agedata2['Age'][ind_b2]])[0] #for Pinsonneault 2018

#remove non-finite entries!
mask2 = np.all(np.isfinite(fullx2), axis=1) & np.all(np.isfinite(fully2), axis=1)
fullx2, fully2 = fullx2[mask2], fully2[mask2]

scaling_x2 = np.median(fullx2, axis=0)
scaling_y2 = np.median(fully2, axis=0)
fullx2, fully2 = fullx2/scaling_x2, fully2/scaling_y2

In [27]:
tenpercent2=len(agedata2['Age'][ind_b2])//10 #figure out what ten percent of this set of age data is

#last name before M 
#trainbin=slice(0,-1*tenpercent-1)
#testing=slice(-1*tenpercent,-1)


#last name M or later
trainbin2=slice(tenpercent2+1,-1)
testing2=slice(0,tenpercent2)


x_train2, y_train2 = fullx2[trainbin2], fully2[trainbin2]
x_test2, y_test2 = fullx2[testing2], fully2[testing2]

In [28]:
model.fit(x_train2, y_train2, epochs=iterations, validation_split=0.05, batch_size=300)

Epoch 1/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 7.9917e-05 - loss: 1.7490 - val_accuracy: 0.0000e+00 - val_loss: 0.2352
Epoch 2/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 7.9917e-05 - loss: 0.2340 - val_accuracy: 0.0000e+00 - val_loss: 0.1936
Epoch 3/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 7.9917e-05 - loss: 0.2043 - val_accuracy: 0.0000e+00 - val_loss: 0.1853
Epoch 4/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 7.9917e-05 - loss: 0.1982 - val_accuracy: 0.0000e+00 - val_loss: 0.1831
Epoch 5/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 7.9917e-05 - loss: 0.1948 - val_accuracy: 0.0000e+00 - val_loss: 0.1820
Epoch 6/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 7.9917e-05 - loss: 0.1930 - val_accuracy: 0.0000e+00 - val_loss: 0.1824
Epoch 7/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 7.9917e-05 - loss: 0.1915 - val_accuracy: 0.0000e+00 - val_loss: 0.1816
Epoch 8/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/s

In [29]:
predictions2 = model.predict(x_test2)
print(len(predictions2))

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
1551


In [30]:
DR19x2 = np.dstack([data_masked['teff'],data_masked['logg'], data_masked['m_h_atm'],
                   data_masked['alpha_m_atm'], data_masked['c_h'], data_masked['n_h']])[0]
print(len(data_masked['teff']))

DR19x2= DR19x2/scaling_x2
predictionsDR19_2 = model.predict(DR19x2)

425631
13301/13301 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step


In [35]:
import pandas as pd
#Pulling Labels and Data
apokasc2_age = predictionsDR19.flatten()
tess_age = predictionsDR19_2.flatten()
age_difference = tess_age - apokasc2_age
star_ids = data_masked['tic_v8_id']
#Creating DataFrame of the results
dataframe = pd.DataFrame({
	"TIC": star_ids,
	'TESS_predicted_Gyr': tess_age,
	'APOKASC2_predicted_Gyr': tess_age,
    'Difference': age_difference
})
#Saving DataFrame to CSV
dataframe.to_csv('TESS_APOKASC2_Comparison.csv', index=False)

In [36]:
print(f'Average difference between data set is:', np.mean(age_difference))

Average difference between data set is: -0.30940393
